# Morse Coefficient Optimization
## By Terry Bondy, VA3TYB

In [1]:
printf(strftime ("Last updated: %A %e %B %Y", localtime (time ())))

Last updated: Sunday  8 December 2019

The notebook concerns itself with optimizing the detection coefficients $\phi_d(i)$ from `MorseCoefficients` notebook where $\phi_d(i)$ is is the $i^{th}$ detection optimizing phase value for the message.

Turns out if $M$ is the number of bits, then only have to optimize $\frac{M}{2}$ values, because for maximal detection, $\phi_d(i) = -\phi_d(M + 1 - i)$


The following function computes a column vector representing the sound/silence coefficients $a_s(i)$ for the message to be sent.

In [2]:
function A_s = makeMorseSoundSilence(message)
    baseCoeff = alphaToMorse(message)';
    A_s = vertcat(baseCoeff, [ 0; 0; 0; 0; 0; 0; 0; ], flip(baseCoeff));
    sz = rows(A_s);
    # See if needs padding
    if (sz < 551)
        half = (551 - sz)/2;
        A_s = vertcat(zeros(half,1), baseCoeff, [ 0; 0; 0; 0; 0; 0; 0; ], flip(baseCoeff), zeros(half,1));
    endif        
endfunction

Try it out

In [3]:
A_s = makeMorseSoundSilence("QRG DE VA3TYB VA3TYB?");
rows(A_s)

ans =  551


Determine the number of "on" bits.

In [4]:
A_s' * A_s

ans =  228


The following is the cost function used to optimize $\phi_d(i)$.

In [5]:
# Phi_d is a column vector, 0 <= phi <= pi, length m
# A_s is a column vector, 0 or 1s, length 2m + 1
function cost = costAny(Phi_d, A_s)
  # Make a test vector where the back half is a mirror and congugate of the front half for maximal detection
  # Centre value is 1, because it is the unit vector that is a congugate of itself
  test = A_s .* [ exp(j .* Phi_d); 1; exp(-j .* flip(Phi_d)) ];
  cost = max(abs(conv(test,flip(test))))/(A_s' * A_s);
endfunction

Try it out. Should get an answer between 0.15 and 0.5 but usually between 0.25 and 0.4.

In [6]:
costAny(pi .* rand(floor(rows(A_s)/2), 1), A_s)

ans =  0.24511


Ensure that you run `Nonlin_min_Install` notebook to install this package.

In [7]:
pkg load optim

Finally a function that will provide $a_s(i)$ and $\phi_d(i)$ for a given message.

In [8]:
function [A_s, Phi_d, objf, cvg] = optimize(message)
  A_s = makeMorseSoundSilence(message);
  m = floor(rows(A_s)/2);
  A_s_half = A_s(1:m);
  init_p = pi .* rand(m, 1) .* A_s_half;
  # Not varying all the parameters
  fixed = not(logical(A_s_half));
  lbound = zeros(m, 1);
  ubound = pi .* ones(m, 1);
  [Phi_d, objf, cvg] = nonlin_min (@ (Phi_trial) costAny(Phi_trial, A_s), init_p, optimset ("Algorithm", "samin", 
    "fixed", fixed,
    "lbound", lbound,
    "ubound", ubound,
    "Display", "iter"
  ));
endfunction

Try it out.

In [ ]:
[VA3ASE_A_s, VA3ASE_Phi_d, objf, cvg] = optimize("QRG DE VA3ASE VA3ASE?");
cvg
objf


temperature no. 1: 1.000000e-01, energy 3.367722e-01,
tries with energy less / not less but accepted / rejected: / to far / new optimum
3688 / 5254 / 258 / 4537 / 7
temperature no. 2: 8.333333e-02, energy 2.919526e-01,
tries with energy less / not less but accepted / rejected: / to far / new optimum
3610 / 5277 / 313 / 4611 / 1
temperature no. 3: 6.944444e-02, energy 2.409643e-01,
tries with energy less / not less but accepted / rejected: / to far / new optimum
3570 / 5280 / 350 / 4583 / 4
temperature no. 4: 5.787037e-02, energy 3.231423e-01,
tries with energy less / not less but accepted / rejected: / to far / new optimum
3539 / 5179 / 482 / 4677 / 0
temperature no. 5: 4.822531e-02, energy 2.897285e-01,
tries with energy less / not less but accepted / rejected: / to far / new optimum
3474 / 5165 / 561 / 4572 / 0
temperature no. 6: 4.018776e-02, energy 2.288672e-01,
tries with energy less / not less but accepted / rejected: / to far / new optimum
3475 / 5042 / 683 / 4615 / 0
temperatur

In [ ]:
[VE3YRA_A_s, VE3YRA_Phi_d, objf, cvg] = optimize("QRG DE VE3YRA VE3YRA?");
cvg
objf

In [ ]:
[VA3TYB_A_s, VA3TYB_Phi_d, objf, cvg] = optimize("QRG DE VA3TYB VA3TYB?");
cvg
objf